# Tasks
## Task 1 : Binary Representations

I will do this using several functions rotate left, rotate right  a choose function and an majority function. Let's start with the rotate left function I want to rotate the bits in a 32 bit unsigned integer too n places n being anything so lets do this one first. 


In [20]:
d = 0xe3


In [21]:
print(d)

227


In [22]:
bin(d)

'0b11100011'

In [23]:
bin(d <<3)

'0b11100011000'

In [24]:
bin(d >> 5)

'0b111'

In [25]:
bin((d << 3) | (d >> 5))

'0b11100011111'

In [26]:
0xb << 3

88

## Task 1a
The number is shifted left by x positions using n << x
The bits that "fall off" the left are caught and moved to the right using n >> (int_bits - x)
The two results are combined using the bitwise OR operator |

In [27]:
import numpy

int_bits = 32

x=2
n=42

def rotl(n ,x):

    return (n << x)|(n >> (int_bits - x))




In [28]:
print("Left Rotation of",n,"by"
      ,x,"is",end=" ")
print(rotl(n, x))

Left Rotation of 42 by 2 is 168


here is some more exmaples to test wether the code is doing what I think it is doing 

In [29]:
#Test example 2
x=16
n=42
print("Left Rotation of",n,"by"
        ,x,"is",end=" ")
print(rotl(n, x))

Left Rotation of 42 by 16 is 2752512


In [30]:
#Test example 3
x=24
n=64
print("Left Rotation of",n,"by"
        ,x,"is",end=" ")
print(rotl(n, x))


Left Rotation of 64 by 24 is 1073741824


## Task 1b
The number is shifted right by x positions using n >> x
The bits that "fall off" the right are caught and moved to the left using n << (int_bits - x)
The two results are combined using the bitwise OR operator |

In [31]:
def rotr(n ,x):

    return (n >> x)|(n << (int_bits - x)) & 0xFFFFFFFF

### Note on Bitwise Operations
The `0xFFFFFFFF` mask in the right rotation function ensures we keep only the 32 least significant bits. In Python, integers have arbitrary precision, so without this mask, left shifts might create values larger than 32 bits when large numbers are rotated.

In [32]:
print("Right Rotation of",n,"by"
      ,x,"is",end=" ")
print(rotr(n, x))

Right Rotation of 64 by 24 is 16384


### Task 1c 
Here we will make a function ch(x, y, z) that chooses the bits from y where x has bits set to 1 and bits in z where x has bits set to 0.



In [33]:
def ch(x, y, z):

    return (x & y) ^ (~x & z)

print(ch(0x1, 0x2, 0x3))

2


The function ch takes three arguments x, y, and z.
It performs a bitwise AND operation between x and y.
It performs a bitwise AND operation between the bitwise NOT of x and z.
It then performs a bitwise XOR operation between the results of the two AND operations.
The result of the XOR operation is returned

### Task1d the maj function 

The function maj(x, y, z) which takes a majority vote of the bits in x, y, and z. The output should have a 1 in bit position i where at least two of x, y, and z have 1's in position i. All other output bit positions should be 0. So this is a simple voting system where the computer decideds on which bit is on or off so to speak for the different bit positions. For each bit position in the paramters it decides if they are yes or no if 2 out of the three are yess the answer is yes if not the answer is no. This implementation follows the Secure Hash Standard (NIST FIPS 180-4) specifications for circular bit rotation operations used in cryptographic hash functions.

I tried to use OR (|) instead of XOR(^) intially but it ultimatly doesnt really matter because of the properties of xor when used in pairs.


In [34]:
def maj(x, y, z):
    return (x & y) ^ (y & z) ^ (z & x)


result = maj(0xe3, 0xf4, 0x5a)
print(result)
print(bin(result))

242
0b11110010


### Implementation Choice
I chose `^` (XOR) instead of `|` (OR) because XOR specifically identifies bits that appear in exactly one of its operands. When taking majority votes across pairs of inputs, the XOR pattern correctly identifies where exactly two out of three bits are set.

# Task 2 : Hash Functions


"The hashing function, which is used by both lookup and install, adds each character value
in the string to a scrambled combination of the previous ones and returns the remainder
modulo the array size. This is not the best possible hash function, but it is short and effective.
 /* hash: form hash value for string s */
 unsigned hash(char *s)
 {
 unsigned hashval;
 for (hashval = 0; *s != '\0'; s++)
 hashval = *s + 31 * hashval;
 129
 return hashval % HASHSIZE;
 }
Unsigned arithmetic ensures that the hash value is non-negative." form the The C programming language by BRIAN W. KERNIGHAN and DENNIS M.RITCHIE

Below is my python implementaion the reason why 31 and 101 were used as they seem as 31 is close to 32 so it is perfect for the masking of asci characters this is also why we use ord to ensure it is ascii characters we are using the ne mod 101 the reason for 101  it is a prime number  is effective like they mention not the best possible hash function but works well.

ord() converts to ASCII (0-127)
Multiplication by 31 provides:
Good bit distribution
CPU-efficient operations
Effective string character mixing
Modulo 101 ensures:
Bounded output range
Even distribution
Minimal collisions

In [35]:
def hash(chars):
    hashval = 0
    for char in chars:
        hashval = ord(char) + 31 * hashval
    return hashval % 101

# Example with my name
result = hash(['O', 'L', 'I', 'V', 'E', 'R'])
print(result)

38


 ### Hash Constant Selection
I chose 31 as the multiplication factor because it's a prime number close to a power of 2 (32), making it efficient for bit-shifting operations while avoiding common patterns that might cause collisions. The modulo 101 provides a good balance between hash space size and collision avoidance. The reasons why these numbers are so good no one knows I suspect it might have something to do with the proximity to 32 and 100 respectively but that is unproven

# Task 3
## Sha256

## Understanding SHA-256 Padding

SHA-256 (Secure Hash Algorithm 256-bit) is a cryptographic hash function that's part of the SHA-2 family designed by the NSA. It produces a 256-bit (32-byte) hash value, typically rendered as a 64-digit hexadecimal number.

### Key Properties of SHA-256:
- **One-way function**: Computationally infeasible to reverse
- **Deterministic**: Same input always produces the same output
- **Avalanche effect**: Small changes in input cause significant changes in output
- **Collision resistance**: Difficult to find two different inputs with the same hash

### Padding Mechanism
SHA-256 operates on blocks of 512 bits. When a message doesn't perfectly fit into these blocks, padding is required. The padding follows a specific format:
1. Append a single '1' bit
2. Append '0' bits until the message length is 64 bits short of a multiple of 512
3. Append the original message length as a 64-bit big-endian integer

This padding ensures that the message is properly structured for the hash algorithm and helps maintain the security properties of the hash function.

In [36]:
def calculate_sha256_padding(file_path):
    # Get the file size by reading the file directly
    with open(file_path, 'rb') as f:
        file_content = f.read()
        file_size = len(file_content)
    
    # Calculate the file size in bits
    file_size_bits = file_size * 8
    
    # Calculate the number of padding bytes needed
    # We need: 1 byte for the '1' bit + zeros + 8 bytes for the length
    # The total length must be a multiple of 64 bytes (512 bits)
    padding_length = 64 - ((file_size + 1 + 8) % 64)
    if padding_length == 64:
        padding_length = 0
    
    # Create the padding
    # First byte: 0x80 (binary 10000000) - the '1' bit followed by 7 zeros
    padding = bytearray([0x80])
    
    # Add the required number of zero bytes
    padding.extend([0x00] * padding_length)
    
    # Add the original message length as a 64-bit big-endian value
    padding.extend(file_size_bits.to_bytes(8, byteorder='big'))
    
    # Format the output as hex bytes with spaces
    padding_hex = ' '.join(f'{b:02x}' for b in padding)
    
    print(f"SHA-256 padding for file '{file_path}':")
    print(padding_hex)
    
    return padding

### Byte Representation Details
The `0x80` byte represents 10000000 in binary - this adds the required '1' bit followed by 7 zeros. This approach lets the system work with whole bytes rather than individual bits, which is more efficient in most programming languages.

# Task 4 : Calculating the First 100 prime numbers


The Sieve of Eratosthenes is one of the oldest algorithms for finding prime numbers, attributed to Eratosthenes of Cyrene, a Greek mathematician who lived in the 3rd century BCE (276-194 BCE). He was the chief librarian at the Library of Alexandria and made significant contributions to mathematics, geography, and astronomy.

Algorithmic Complexity
Time Complexity: O(n log log n)
This is better than the naive O(n²) approach
The inner loop executes approximately n/2 + n/3 + n/5 + ... times, which evaluates to O(n log log n)
Space Complexity: O(n)
Requires a boolean array of size n+1
Programming Perspective

Advantages:
- Very efficient for generating all primes up to a certain limit
- Simple to implement and understand
- Optimal asymptotic complexity for finding all primes in a range

Disadvantages:
- Memory-intensive for large values of n
- Not suitable when you only need to check if a single number is prime
- Not easily parallelizable in its basic form

I picked this primarily because I liked the name and I think its good idea to have a older inefficent one for context. 

In [37]:
# Algoritim 1 : Sieve of Eratosthenes
def sieve_of_eratosthenes(n):
    """
    Find all prime numbers up to n using the Sieve of Eratosthenes.
    Returns a list of prime numbers.
    """
    # Initialize the sieve array with True values
    sieve = [True] * (n + 1)
    # 0 and 1 are not prime
    sieve[0] = sieve[1] = False
    
    # Check numbers from 2 to sqrt(n)
    for i in range(2, int(n**0.5) + 1):
        # If i is prime (not marked false yet)
        if sieve[i]:
            # Mark all multiples of i as non-prime
            for j in range(i*i, n + 1, i):
                sieve[j] = False
    
    # Collect all prime numbers from the sieve
    primes = [i for i in range(2, n + 1) if sieve[i]]
    return primes

# Find enough prime numbers to ensure we get at least 100
first_100_primes = sieve_of_eratosthenes(550)[:100]
print("First 100 primes (Sieve of Eratosthenes):")
print(f"First 5 primes: {first_100_primes[:5]}")
print(f"Last 5 primes: {first_100_primes[-5:]}")
print(f"Count: {len(first_100_primes)}")



First 100 primes (Sieve of Eratosthenes):
First 5 primes: [2, 3, 5, 7, 11]
Last 5 primes: [503, 509, 521, 523, 541]
Count: 100


### Sieve Optimisation
I optimised the sieve implementation by starting the inner loop at i*i instead of 2*i, since all multiples of i less than i*i would have already been marked by considering smaller primes. This reduces the number of unnecessary operations.

# Trial Division
## Historical Background
Trial division is the most basic primality test and has been known since ancient times. It doesn't have a specific inventor as it follows directly from the definition of prime numbers. The optimization of checking only up to the square root comes from ancient Greek mathematics, and the further optimization of checking only numbers of form 6k±1 is a more modern enhancement.

Algorithmic Complexity
Time Complexity:

For checking a single number n: O(√n)
For finding first m primes: O(m² log m) approximately, because:
The mth prime is approximately m log m (by the Prime Number Theorem)
Each primality check takes O(√n) time
This makes it less efficient than the Sieve for finding multiple primes
Space Complexity: O(1) for checking primality, O(m) for storing m primes

Programming Perspective
Advantages:
- Simple to understand and implement
- Memory-efficient when checking individual numbers
- Works well for sparse primality testing
- Can be stopped as soon as a desired count is reached

Disadvantages:
- Inefficient for finding large quantities of primes
- Performance heavily depends on optimization techniques
- Becomes very slow for large numbers

In [38]:
# Algorithm 2 : Trial Division
def is_prime(n):
    """Check if a number is prime using trial division."""
    if n <= 1:
        return False
    if n <= 3:
        return True
    if n % 2 == 0 or n % 3 == 0:
        return False
    
    # Check divisibility by numbers of form 6k ± 1
    i = 5
    while i * i <= n:
        if n % i == 0 or n % (i + 2) == 0:
            return False
        i += 6
    return True

def find_first_n_primes(n):
    """Find the first n prime numbers using trial division."""
    primes = []
    num = 2
    while len(primes) < n:
        if is_prime(num):
            primes.append(num)
        num += 1
    return primes

first_100_primes_trial = find_first_n_primes(100)
print("\nFirst 100 primes (Trial Division):")
print(f"First 5 primes: {first_100_primes_trial[:5]}")
print(f"Last 5 primes: {first_100_primes_trial[-5:]}")
print(f"Count: {len(first_100_primes_trial)}")


First 100 primes (Trial Division):
First 5 primes: [2, 3, 5, 7, 11]
Last 5 primes: [503, 509, 521, 523, 541]
Count: 100


# Task 5 : Roots

## Bit Representation of Square Roots

This task explores the binary representation of the fractional parts of square roots, particularly of prime numbers. This has applications in:

- **Random number generation**: Square roots of primes produce high-quality pseudo-random sequences
- **Cryptographic constants**: Many cryptographic algorithms use these values as initialisation vectors
- **Number theory**: Examining patterns in these bit sequences provides insights into irrational numbers

### Method Used
The system extracts the first 32 bits of the fractional part of square roots by:
1. Computing the square root using standard mathematical libraries
2. Removing the integer part to isolate the fractional component
3. Repeatedly multiplying by 2 and checking if the result exceeds 1
4. Building a binary string bit by bit based on this process

This is effectively the same algorithm used to convert decimal fractions to binary, applied specifically to the fractional parts of square roots.

### Significance
The fractional parts of square roots of different numbers, especially primes, appear to be randomly distributed. This property makes them useful as sources of "nothing-up-my-sleeve numbers" - constants that are derived from a simple, well-defined process rather than arbitrarily chosen.

In [39]:
import math

def get_fractional_bits_direct(number, bits=32):
    """
    Calculate the first 32 bits of the fractional part of the square root using the direct method.
    Returns the integer representation of those bits.
    """
    # Calculate square root
    sqrt_value = math.sqrt(number)
    
    # Get just the fractional part using modulo 1
    fractional_part = sqrt_value % 1
    
    # Shift bits in front of decimal point and convert to integer
    bits_int = int(fractional_part * (2 ** bits))
    
    return bits_int

# Use existing primes from the sieve function
first_100_primes = sieve_of_eratosthenes(550)[:100]

# Calculate and display the results
print("Square roots of the first 100 primes (direct calculation method):")
print("-" * 80)
print("Prime | Square Root | First 32 bits of fractional part | Hex representation")
print("-" * 80)

for prime in first_100_primes:
    # Get integer representation of the bits
    bits_int = get_fractional_bits_direct(prime)
    
    # Format as binary and hex strings
    binary_str = format(bits_int, '032b')
    hex_representation = format(bits_int, '08x')
    sqrt_value = math.sqrt(prime)
    
    print(f"{prime:5d} | {sqrt_value:.8f} | {binary_str} | 0x{hex_representation}")

print("-" * 80)
print(f"Total primes processed: {len(first_100_primes)}")

Square roots of the first 100 primes (direct calculation method):
--------------------------------------------------------------------------------
Prime | Square Root | First 32 bits of fractional part | Hex representation
--------------------------------------------------------------------------------
    2 | 1.41421356 | 01101010000010011110011001100111 | 0x6a09e667
    3 | 1.73205081 | 10111011011001111010111010000101 | 0xbb67ae85
    5 | 2.23606798 | 00111100011011101111001101110010 | 0x3c6ef372
    7 | 2.64575131 | 10100101010011111111010100111010 | 0xa54ff53a
   11 | 3.31662479 | 01010001000011100101001001111111 | 0x510e527f
   13 | 3.60555128 | 10011011000001010110100010001100 | 0x9b05688c
   17 | 4.12310563 | 00011111100000111101100110101011 | 0x1f83d9ab
   19 | 4.35889894 | 01011011111000001100110100011001 | 0x5be0cd19
   23 | 4.79583152 | 11001011101110111001110101011101 | 0xcbbb9d5d
   29 | 5.38516481 | 01100010100110100010100100101010 | 0x629a292a
   31 | 5.56776436 | 10010

### Bit Extraction Method
The bit extraction algorithm mimics how decimal-to-binary conversion works for fractions: repeatedly multiply by 2, take the integer part as the next bit, and continue with the remaining fractional part. This lets us extract bits without relying on floating-point precision issues.

# Task 6 : Proof of Work

Find the word(s) in the English language with the greatest number of 0 bits at the beginning of their SHA256 hash digest.
Include proof that any word you list is in at least one English dictionary.

To solve this problem  I need to do the following 
- compute the SHA-256 hash of english words
- count the number of leading zero bits in the hash 
- Find the words with the maximum amount of leading zeros bits

### Blockchain Connection: Square Roots as Cryptographic Constants
The binary representation of square roots of prime numbers plays a crucial role in blockchain technology. Bitcoin's SHA-256 implementation uses similar mathematical constants (cube roots of primes) as initialisation vectors and round constants, ensuring these values are provably not manipulated. These "nothing-up-my-sleeve" numbers provide a cryptographic foundation that all network participants can independently verify without trusting any central authority. When Bitcoin's creator needed constants for the hash function, using mathematical derivations from irrational roots ensured transparency, as their values come from mathematics rather than arbitrary choice. This application demonstrates how pure mathematical concepts form the foundation of trustless consensus systems.

In [40]:
import hashlib
import os

def count_leading_zero_bits(hash_bytes):
    """Count the number of leading zero bits in a hash byte string"""
    count = 0
    for byte in hash_bytes:
        if byte == 0:
            count += 8
        else:
            # Count leading zeros in this non-zero byte
            bits = bin(byte)[2:].zfill(8)
            count += bits.index('1')
            break
    return count

def find_words_with_most_leading_zeros(dictionary_path='/workspaces/computational_theory/resources/words.txt'):
    """Find English words with the most leading zero bits in their SHA-256 hash."""
    max_zeros = 0
    best_words = []
    processed = 0
    
    print(f"Using dictionary at: {dictionary_path}")
    
    with open(dictionary_path, 'r', encoding='utf-8', errors='ignore') as f:
        for word in f:
            word = word.strip()
            if not word:
                continue
            
            processed += 1
            if processed % 10000 == 0:
                print(f"Processed {processed} words...")
            
            hash_bytes = hashlib.sha256(word.encode('utf-8')).digest()
            zeros = count_leading_zero_bits(hash_bytes)
            
            if zeros > max_zeros:
                max_zeros = zeros
                best_words = [word]
            elif zeros == max_zeros:
                best_words.append(word)
    
    return max_zeros, best_words, processed

# Run with path to the local dictionary file
dictionary_path = '/workspaces/computational_theory/resources/words.txt'
max_zeros, best_words, total_processed = find_words_with_most_leading_zeros(dictionary_path)


print(f"\nWords with the most leading zero bits:")
print("-" * 30)
for word in best_words:
    print(f"Word: {word} | Leading zeros: {max_zeros}")
print("-" * 30)
print(f"Total words processed: {total_processed}")
print(f"Dictionary: {dictionary_path}")

Using dictionary at: /workspaces/computational_theory/resources/words.txt
Processed 10000 words...
Processed 20000 words...
Processed 30000 words...
Processed 40000 words...

Words with the most leading zero bits:
------------------------------
Word: APPLICANT | Leading zeros: 16
------------------------------
Total words processed: 45402
Dictionary: /workspaces/computational_theory/resources/words.txt


### Efficient Zero Counting
My implementation counts leading zeros in two phases: first checking for whole bytes of zeros (8 bits at a time) for efficiency, then falling back to bit-by-bit counting only when needed. This significantly speeds up processing for hashes with many leading zeros.

As we can see from the results the word in words.txt taken from the emerging tech mdoule  with the most leading zeros is "APPLICANT" at 16 leading zeros

# Task 7: Turing Machine


### Binary Increment Turing Machine

This Turing machine adds 1 to a binary number on its tape:

1. **State S (Scan)**: Starts at the leftmost digit and scans right until reaching the end of the input
2. **State C (Carry)**: Works backward from right to left:
   - Changes 1s to 0s (carrying the 1)
   - When it finds a 0, changes it to 1 and halts
   - If all digits are 1s, adds a new 1 digit at the front
3. **State H (Halt)**: Final state reached when addition is complete

This implementation correctly handles all cases, including:
- Regular addition when there's a 0 digit to change
- Overflow cases where all digits are 1s (e.g., 1111 → 10000)
- Empty or single-digit inputs

The machine demonstrates one of the fundamental operations in computing - binary counting - using only three states and simple transition rules.

## Reference

3. Turing, A. M. (1936). "On Computable Numbers, with an Application to the Entscheidungsproblem." Proceedings of the London Mathematical Society, Series 2, Volume 42, Issue 1, pp. 230-265.https://www.cs.virginia.edu/~robins/Turing_Paper_1936.pdf

In [41]:
# Define a Turing machine for binary increment
Q = {'S', 'C', 'H'}  # States: Scan, Carry, Halt
T = {'0', '1', '_'}  # Alphabet
b = '_'              # Blank symbol
A = {'0', '1'}       # Input alphabet

# Transition function with the state table
state_table = {
    # Scan state - move right until reaching the end
    ('S', '0'): ('S', '0', 'R'),  # Stay in S, keep 0, move right
    ('S', '1'): ('S', '1', 'R'),  # Stay in S, keep 1, move right
    ('S', '_'): ('C', '_', 'L'),  # End reached, move to Carry state, go left
    
    # Carry state - move left, flip bits as needed
    ('C', '0'): ('H', '1', 'N'),  # Found a 0, change to 1, halt
    ('C', '1'): ('C', '0', 'L'),  # Found a 1, change to 0, continue carrying left
    ('C', '_'): ('H', '1', 'R'),  # Reached left end with all 1s, add new 1, halt
}

s = 'S'  # Initial state
F = {'H'}  # Final/halt state

def simulate(table, initial_input, verbose=True, max_steps=1000):
    """Simulate a Turing machine with safeguards against infinite loops."""
    if verbose:
        print(f'Length of input: {len(initial_input)}')
    
    # Keep track of number of operations
    no_ops = 0

    # Check for empty input
    if initial_input == '':
        initial_input = '_'
    
    # Make input a list
    tape = list(initial_input)
    
    # Initial position on the tape
    pos = 0

    # Use the defined initial state directly
    state = s  # Use the predefined initial state
    
    if verbose:
        # Print starting config
        print(state + ''.join(tape))

    # While not in a final state and within step limit
    while state not in F and no_ops < max_steps:
        try:
            # Get current symbol
            symbol = tape[pos]
            
            # Get next state, symbol, and direction
            state, tape[pos], direction = table[(state, symbol)]
            
            # Increment number of operations
            no_ops += 1
            
            # Move cell based on direction
            if direction == 'L':
                pos -= 1
            elif direction == 'R':
                pos += 1
            # 'N' (no movement) is handled implicitly
            
            # Fix problems with Python lists being finite
            if pos < 0:
                tape = [b] + tape
                pos = 0
            if pos == len(tape):
                tape.append('_')

            if verbose:
                # Print current configuration
                print(''.join(tape[:pos]) + state + ''.join(tape[pos:]))
                
        except KeyError:
            if verbose:
                print(f"Error: No transition defined for state '{state}' and symbol '{symbol}'")
            break

    if no_ops >= max_steps and verbose:
        print("Warning: Maximum steps reached, possible infinite loop")
        
    if verbose:
        # Print final tape content without the state marker
        print(f'Final result: {"".join(tape).rstrip("_")}')
        print(f'Number of operations: {no_ops}')
    
    return state, ''.join(tape).rstrip('_')

# Test examples
print("Testing Binary Increment Machine")
print("\nExample 1: Adding 1 to 100111 (39 decimal)")
final_state, result = simulate(state_table, '100111')
print(f"Expected: 101000, Result: {result}")

print("\nExample 2: Adding 1 to 1111 (15 decimal)")
final_state, result = simulate(state_table, '1111')
print(f"Expected: 10000, Result: {result}")

print("\nExample 3: Adding 1 to 0 (0 decimal)")
final_state, result = simulate(state_table, '0')
print(f"Expected: 1, Result: {result}")

Testing Binary Increment Machine

Example 1: Adding 1 to 100111 (39 decimal)
Length of input: 6
S100111
1S00111
10S0111
100S111
1001S11
10011S1
100111S_
10011C1_
1001C10_
100C100_
10C0000_
10H1000_
Final result: 101000
Number of operations: 11
Expected: 101000, Result: 101000

Example 2: Adding 1 to 1111 (15 decimal)
Length of input: 4
S1111
1S111
11S11
111S1
1111S_
111C1_
11C10_
1C100_
C1000_
C_0000_
1H0000_
Final result: 10000
Number of operations: 10
Expected: 10000, Result: 10000

Example 3: Adding 1 to 0 (0 decimal)
Length of input: 1
S0
0S_
C0_
H1_
Final result: 1
Number of operations: 3
Expected: 1, Result: 1


### State Transition Design
The Turing machine state transitions follow a specific pattern: S (Scan) moves right until it reaches the end of input, C (Carry) works backward flipping 1s to 0s while looking for a 0 to change to 1, and H (Halt) is the final state. This models the carry operation in binary addition perfectly.

# Task 8: Computational Complexity

## Computational Complexity Analysis of Sorting Algorithms

### Bubble Sort Analysis

#### Time Complexity
- **Worst-case:** O(n²) - Occurs when the array is in reverse order
- **Average-case:** O(n²) - Even with random data, we expect quadratic time
- **Best-case:** O(n) - Occurs when the array is already sorted (with the early termination optimisation)

#### Space Complexity
- O(1) - Bubble sort is an in-place algorithm that only requires a constant amount of additional memory regardless of input size

#### Observations from the implementation
The bubble sort implementation with different permutations of [1,2,3,4,5] demonstrates this complexity. The number of comparisons varies from 7 (for nearly sorted arrays) to 10 (for reverse or highly disordered arrays), showing how the algorithm's performance depends on the initial order of elements.

### Better Sorting Algorithms

#### Merge Sort
- **Time Complexity:** O(n log n) in all cases
- **Space Complexity:** O(n) - Requires additional memory proportional to the input size
- **Advantages:** Stable, predictable performance regardless of input distribution
- **Disadvantages:** Not in-place, requires extra memory

#### Quick Sort
- **Time Complexity:** O(n log n) average, O(n²) worst case
- **Space Complexity:** O(log n) average due to recursion stack
- **Advantages:** Excellent performance in practice, can be implemented in-place
- **Disadvantages:** Unstable, worst-case performance on already sorted arrays (can be mitigated)



### Algorithm Selection Considerations

The choice of sorting algorithm depends on several factors:
- **Input size:** For small arrays (n < 10-20), simpler algorithms like insertion sort can outperform asymptotically better algorithms
- **Nearly sorted data:** Insertion sort or bubble sort with early termination excel here
- **Memory constraints:** Heap sort or in-place quicksort are preferable when memory is limited
- **Stability requirement:** If maintaining relative order of equal elements matters, choose merge sort or insertion sort
- **Guaranteed performance:** When worst-case performance matters, avoid quicksort and choose merge sort or heap sort

### Conclusion
While the bubble sort implementation works for small datasets, it would be impractical for larger arrays due to its O(n²) complexity. In a production environment, most programming languages provide optimised sorting functions that typically implement adaptive strategies, often using hybrid algorithms like Timsort (Python, Java) that combine the strengths of different sorting approaches

In [42]:
import itertools

def bubble_sort_with_stats(arr):
    """Perform bubble sort and return show metrics."""
    arr_copy = arr.copy()
    n = len(arr_copy)
    comparisons = 0
    swaps = 0
    
    for i in range(n):
        is_sorted = True
        for j in range(n-i-1):
            comparisons += 1
            if arr_copy[j] > arr_copy[j+1]:
                arr_copy[j], arr_copy[j+1] = arr_copy[j+1], arr_copy[j]
                swaps += 1
                is_sorted = False
        if is_sorted:
            break
            
    return comparisons, swaps

# Create all permutations of [1,2,3,4,5]
default = [1,2,3,4,5]
permutations = list(itertools.permutations(default))

# Test each permutation
for perm in permutations:
    perm_list = list(perm)
    comparisons, swaps = bubble_sort_with_stats(perm_list)
    print(f"Permutation: {perm_list} → Comparisons: {comparisons}, Swaps: {swaps}")

Permutation: [1, 2, 3, 4, 5] → Comparisons: 4, Swaps: 0
Permutation: [1, 2, 3, 5, 4] → Comparisons: 7, Swaps: 1
Permutation: [1, 2, 4, 3, 5] → Comparisons: 7, Swaps: 1
Permutation: [1, 2, 4, 5, 3] → Comparisons: 9, Swaps: 2
Permutation: [1, 2, 5, 3, 4] → Comparisons: 7, Swaps: 2
Permutation: [1, 2, 5, 4, 3] → Comparisons: 9, Swaps: 3
Permutation: [1, 3, 2, 4, 5] → Comparisons: 7, Swaps: 1
Permutation: [1, 3, 2, 5, 4] → Comparisons: 7, Swaps: 2
Permutation: [1, 3, 4, 2, 5] → Comparisons: 9, Swaps: 2
Permutation: [1, 3, 4, 5, 2] → Comparisons: 10, Swaps: 3
Permutation: [1, 3, 5, 2, 4] → Comparisons: 9, Swaps: 3
Permutation: [1, 3, 5, 4, 2] → Comparisons: 10, Swaps: 4
Permutation: [1, 4, 2, 3, 5] → Comparisons: 7, Swaps: 2
Permutation: [1, 4, 2, 5, 3] → Comparisons: 9, Swaps: 3
Permutation: [1, 4, 3, 2, 5] → Comparisons: 9, Swaps: 3
Permutation: [1, 4, 3, 5, 2] → Comparisons: 10, Swaps: 4
Permutation: [1, 4, 5, 2, 3] → Comparisons: 9, Swaps: 4
Permutation: [1, 4, 5, 3, 2] → Comparisons: 1

### Early Termination Optimization
I implemented the "early termination" optimisation in bubble sort using the `is_sorted` flag, which allows the algorithm to stop once a pass makes no swaps. This significantly improves performance for nearly-sorted arrays, changing the best-case time complexity from O(n²) to O(n).

# References
1. National Institute of Standards and Technology (2015). FIPS PUB 180-4: Secure Hash Standard (SHS). Section 2.2.2: Elementary Functions.https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf  
2. The C programming Lanaguage by DENNIS M.RITCHIE and BRIAN W. KERNIGHAN
https://seriouscomputerist.atariverse.com/media/pdf/book/C%20Programming%20Language%20-%202nd%20Edition%20(OCR).pdf  
3. Turing, A. M. (1936). "On Computable Numbers, with an Application to the Entscheidungsproblem." Proceedings of the London Mathematical Society, Series 2, Volume 42, Issue 1, pp. 230-265.https://www.cs.virginia.edu/~robins/Turing_Paper_1936.pdf
4. Bubble sort explainationw3schools. https://www.w3schools.com/dsa/dsa_algo_bubblesort.php#:~:text=The%20Bubble%20Sort%20algorithm%20loops,again%20and%20again%20n%20times.
5. Sieve of Eratones code : https://www.geeksforgeeks.org/python-program-for-sieve-of-eratosthenes/
6. Trail Divison :https://www.geeksforgeeks.org/trial-division-algorithm-for-prime-factorization/
7. Root cubes help :https://github.com/ianmcloughlin/computational_theory/blob/main/materials/cube_roots.ipynb